In [74]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,VotingRegressor
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor




In [3]:
hr = pd.read_csv("HR_comma_sep.csv")

In [5]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14995 entries, 0 to 14994
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14995 non-null  float64
 1   last_evaluation        14995 non-null  float64
 2   number_project         14995 non-null  int64  
 3   average_montly_hours   14995 non-null  int64  
 4   time_spend_company     14995 non-null  int64  
 5   Work_accident          14995 non-null  int64  
 6   left                   14995 non-null  int64  
 7   promotion_last_5years  14995 non-null  int64  
 8   Department             14995 non-null  object 
 9   salary                 14995 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [7]:
X = hr.drop('left',axis= 1)
y = hr['left']

In [52]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3)

In [11]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')

scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()



In [13]:
dtc = DecisionTreeClassifier(random_state= 24,max_depth = 2)
lr = LogisticRegression(random_state= 24, C = 0.4)
nb = GaussianNB()

In [15]:
voting = VotingClassifier([('DT',dtc),('LR',lr),('NB',nb)],voting = 'soft') # default voting = hard

In [17]:
pipe = Pipeline([('CT',ct_ohe),('SCL',None),('VT',voting)])

In [31]:
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001E5BD4313A0>),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001E5BD432150>)],
                     verbose_feature_names_out=False)),
  ('SCL', None),
  ('VT',
   VotingClassifier(estimators=[('DT',
                                 DecisionTreeClassifier(max_depth=2,
                                                        random_state=24)),
                                ('LR', LogisticRegression(C=0.4, random_state=24)),
                                ('NB', GaussianNB())],
                    voting='soft'))],
 've

In [23]:
params = {'VT__DT__min_samples_split':[2,10,20],
          'VT__DT__min_samples_leaf':[1,10,20],
          'VT__DT__max_depth':[None,2,3],
          'VT__LR__C': np.linspace(0.001,3,5)
         }
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 24)
gcv = GridSearchCV(pipe,param_grid = params,scoring = 'roc_auc',
                   cv = kfold, verbose = 3)

In [25]:
gcv.fit(X,y)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__LR__C=0.001;, score=0.981 total time=   0.2s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__LR__C=0.001;, score=0.979 total time=   0.2s


...

[CV 2/5] END VT__DT__max_depth=3, VT__DT__min_samples_leaf=20, VT__DT__min_samples_split=20, VT__LR__C=3.0;, score=0.945 total time=   0.1s
[CV 3/5] END VT__DT__max_depth=3, VT__DT__min_samples_leaf=20, VT__DT__min_samples_split=20, VT__LR__C=3.0;, score=0.947 total time=   0.1s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 4/5] END VT__DT__max_depth=3, VT__DT__min_samples_leaf=20, VT__DT__min_samples_split=20, VT__LR__C=3.0;, score=0.943 total time=   0.1s
[CV 5/5] END VT__DT__max_depth=3, VT__DT__min_samples_leaf=20, VT__DT__min_samples_split=20, VT__LR__C=3.0;, score=0.948 total time=   0.1s


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001E5BD4313A0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._colu...
                                                                      DecisionTreeClassifier(max_depth=2,
                                                                                             random_state=24)),
                                                                     ('LR',
                                                                      LogisticRegression(C=0.4,
                                                                                         random_state=24)),
                                                                     ('NB',
                                                                      GaussianNB())],
                                                         voting='soft'))]),
             param_grid={'VT__DT__max_depth': [None, 2, 3],
                         'VT__DT__min_samples_leaf': [1, 10, 20],
                         'VT__DT__min_samples_split': [2, 10, 20],
                         'VT__LR__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00])},
             scoring='roc_auc', verbose=3)

In [27]:
print(gcv.best_params_)
print(gcv.best_score_)

{'VT__DT__max_depth': None, 'VT__DT__min_samples_leaf': 1, 'VT__DT__min_samples_split': 2, 'VT__LR__C': 0.001}
0.980087913237963


In [29]:
model = gcv.best_estimator_
model.fit(X_train,y_train)
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9817226896482176

In [42]:
df_con = pd.read_csv('Concrete_Data.csv')

In [44]:
df_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Cement            1030 non-null   float64
 1   Blast             1030 non-null   float64
 2   Fly               1030 non-null   float64
 3   Water             1030 non-null   float64
 4   Superplasticizer  1030 non-null   float64
 5   Coarse            1030 non-null   float64
 6   Fine              1030 non-null   float64
 7   Age               1030 non-null   int64  
 8   Strength          1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [126]:
X = df_con.drop('Strength',axis = 1)
y = df_con['Strength']

In [128]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3)

In [132]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
r2_lr = r2_score(y_test,y_pred)
r2_lr

0.5771752777048791

In [134]:
el = ElasticNet(random_state = 24)
el.fit(X_train,y_train)
y_pred = el.predict(X_test)
r2_el = r2_score(y_test,y_pred)
r2_el

0.5766806310401154

In [136]:
dtc = DecisionTreeRegressor(random_state = 24)
dtc.fit(X_train,y_train)
y_pred = dtc.predict(X_test)
r2_dtr = r2_score(y_test,y_pred)
r2_dtr

0.83089474226832

In [91]:
voting = VotingRegressor([('LR',lr),('EL',el),('DT',dtc)])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test,y_pred)

0.7641542071921111

##Weighted Average

In [97]:
voting = VotingRegressor([('LR',lr),('EL',el),('DT',dtc)],
                        weights = [0.15,0.15,.7])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.8516273862659319

## OR

In [94]:
voting = VotingRegressor([('LR',lr),('EL',el),('DT',dtc)],
                        weights = [r2_lr,r2_el,r2_dtr])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.7955821089551822

## Voting Using estimators of the same algorithm

In [101]:
dtr1 = DecisionTreeRegressor(random_state = 24, max_depth = None)
dtr2 = DecisionTreeRegressor(random_state = 24, max_depth = 3)
dtr3 = DecisionTreeRegressor(random_state = 24, max_depth = 4)


voting = VotingRegressor([('M1',dtr1),('M2',dtr2),('M3',dtr3)],
                        weights = [0.7,0.15,0.15])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.8436797107172187